In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_dct import dct_2d
from PIL import Image
import torchvision.transforms as transforms
import timm

C:\Users\trung\miniconda3\envs\image_forgery\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [96]:
def log_magnitude(f_shift):
    return torch.log(1 + torch.abs(f_shift))

class ImageToFrequency(nn.Module):
    def __init__(self):
        super(ImageToFrequency, self).__init__()

    def forward(self, x):
        x = dct_2d(x, norm='')
        x = torch.fft.fftshift(x)
        x = log_magnitude(x)
        return x

In [100]:
image = Image.open(r'D:\image_forgery_detection\data\CASIA1\train\fake\Sp_S_NNN_T_txt0068_txt0068_0068.jpg')
image = transforms.ToTensor()(image)
image_2 = image * 255.0

In [3]:
model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)
model

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (drop1): 

In [5]:
model.layers[0]

SwinTransformerStage(
  (downsample): Identity()
  (blocks): Sequential(
    (0): SwinTransformerBlock(
      (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (attn): WindowAttention(
        (qkv): Linear(in_features=96, out_features=288, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=96, out_features=96, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
        (softmax): Softmax(dim=-1)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=96, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path2): Identity()
    )
    (1): SwinTransformerBlock(
      (norm1): LayerNorm((96,)